In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.flows.base import Flow
from nflows.transforms.dropout import UniformStochasticDropout
from nflows.transforms.dropout import VariationalStochasticDropout
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform

In [4]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [5]:
# This works with any size x
def p(x, n_probs):
    sums = torch.sum(x, axis=1)
    probs = torch.cos(torch.ger(sums, torch.arange(1, n_probs+1, dtype=torch.float32)))**2
    norm = torch.sum(probs, axis=1)

    for i in range(n_probs):
        probs[:,i] /= norm
    
    return probs

In [6]:
def generate(n, drop_indices):
    n_probs = torch.max(drop_indices) + 1
    x = torch.rand(n, drop_indices.shape[0])
    probs = p(x, n_probs)

    # Pick a prob
    probs_cumsum = torch.cumsum(probs, axis=1)

    # Tensor with bools that are true when r passes the cumprob
    larger_than_cumprob = torch.rand(n,1) < probs_cumsum
    # Do the arange trick to find first nonzero
    # This is the HIGHEST LABEL FROM DROP_INDICES THAT IS KEPT
    selected_index = torch.argmax(larger_than_cumprob*torch.arange(n_probs, 0, -1), axis=1)

    '''
    print("The index of the selected probability")
    print("This is also the highest label in drop_indices that is kept")
    print(selected_index)
    ''' 
    
    # Find the index of the first true
    drop_mask = drop_indices > selected_index[:,None]
    x[drop_mask] = 0
    
    return x

In [7]:
drop_indices = torch.tensor([0,0,1,1,1,2,3,3,4])
n_data = int(1e6)
x_data = generate(n_data, drop_indices).to(device)

In [8]:
num_layers = 6
base_dist_uniform = BoxUniform(torch.zeros(drop_indices.shape[0]), torch.ones(drop_indices.shape[0]))
base_dist_variational = BoxUniform(torch.zeros(drop_indices.shape[0]), torch.ones(drop_indices.shape[0]))

transforms_uniform = []
transforms_variational = []

transforms_uniform.append(UniformStochasticDropout(drop_indices))
transforms_variational.append(VariationalStochasticDropout(drop_indices))

for _ in range(num_layers):
    transforms_uniform.append(RandomPermutation(features=drop_indices.shape[0]))
    transforms_uniform.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=drop_indices.shape[0], 
        hidden_features=50,
        num_bins=10,
        num_blocks=4,
    ))

    transforms_variational.append(RandomPermutation(features=drop_indices.shape[0]))
    transforms_variational.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=drop_indices.shape[0], 
        hidden_features=50,
        num_bins=10,
        num_blocks=4,
    ))

transform_uniform = CompositeTransform(transforms_uniform)
transform_variational = CompositeTransform(transforms_variational)

flow_uniform = Flow(transform_uniform, base_dist_uniform).to(device)
flow_variational = Flow(transform_variational, base_dist_variational).to(device)

optimizer_uniform = optim.Adam(flow_uniform.parameters())
optimizer_variational = optim.Adam(flow_variational.parameters())

In [9]:
n_epochs = 200
batch_size = 10000
n_batches = m.ceil(x_data.shape[0]/batch_size)

for epoch in range(n_epochs):
    permutation = torch.randperm(x_data.shape[0], device=device)    

    # Loop over batches
    cum_loss_uniform = 0
    cum_loss_variational = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, x_data.shape[0]-1 )
        indices = permutation[batch_begin:batch_end]
        batch_x = x_data[indices]
        
        # Take a step
        optimizer_uniform.zero_grad()
        optimizer_variational.zero_grad()

        loss_uniform = -flow_uniform.log_prob(inputs=batch_x).mean()
        loss_variational = -flow_variational.log_prob(inputs=batch_x).mean()

        loss_uniform.backward()
        loss_variational.backward()

        optimizer_uniform.step()
        optimizer_variational.step()

        # Compute cumulative loss
        cum_loss_uniform = (cum_loss_uniform*batch + loss_uniform.item())/(batch+1)
        cum_loss_variational = (cum_loss_variational*batch + loss_variational.item())/(batch+1)

        print("epoch = ", epoch, "batch = ",batch+1, "/", n_batches, "loss_uniform = ", cum_loss_uniform, " loss_variational = ", cum_loss_variational)

/workspace/nflows/nflows/transforms/dropout.py:85: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  log_probs_dropout_selected = torch.log(F.softmax(self._weights)[probs_dropout_index])


epoch =  0 batch =  1 / 1000 loss_uniform =  4.152677059173584  loss_variational =  6.120657920837402
epoch =  0 batch =  2 / 1000 loss_uniform =  4.094895124435425  loss_variational =  5.9657301902771
epoch =  0 batch =  3 / 1000 loss_uniform =  3.9773677984873452  loss_variational =  5.8011555671691895
epoch =  0 batch =  4 / 1000 loss_uniform =  3.885342299938202  loss_variational =  5.615455746650696
epoch =  0 batch =  5 / 1000 loss_uniform =  3.818865489959717  loss_variational =  5.487174034118652
epoch =  0 batch =  6 / 1000 loss_uniform =  3.7399772802988687  loss_variational =  5.333377122879028
epoch =  0 batch =  7 / 1000 loss_uniform =  3.675703832081386  loss_variational =  5.201341833387103
epoch =  0 batch =  8 / 1000 loss_uniform =  3.610759735107422  loss_variational =  5.082662045955658
epoch =  0 batch =  9 / 1000 loss_uniform =  3.5531297789679632  loss_variational =  4.984467294481066
epoch =  0 batch =  10 / 1000 loss_uniform =  3.492845368385315  loss_variationa

epoch =  0 batch =  81 / 1000 loss_uniform =  2.3266981634092923  loss_variational =  2.886233635890631
epoch =  0 batch =  82 / 1000 loss_uniform =  2.3210371063976756  loss_variational =  2.8754255902476427
epoch =  0 batch =  83 / 1000 loss_uniform =  2.315337341952037  loss_variational =  2.8648890343057105
epoch =  0 batch =  84 / 1000 loss_uniform =  2.3096360493273966  loss_variational =  2.8552088127249764
epoch =  0 batch =  85 / 1000 loss_uniform =  2.3046014252830958  loss_variational =  2.845240759849548
epoch =  0 batch =  86 / 1000 loss_uniform =  2.299064548902734  loss_variational =  2.8354436145272364
epoch =  0 batch =  87 / 1000 loss_uniform =  2.2936447310721744  loss_variational =  2.82580261120851
epoch =  0 batch =  88 / 1000 loss_uniform =  2.2883881086652935  loss_variational =  2.8162853392687706
epoch =  0 batch =  89 / 1000 loss_uniform =  2.2831771735394946  loss_variational =  2.807171682293495
epoch =  0 batch =  90 / 1000 loss_uniform =  2.27812335226270

epoch =  0 batch =  161 / 1000 loss_uniform =  2.047660806904669  loss_variational =  2.3900273601460893
epoch =  0 batch =  162 / 1000 loss_uniform =  2.0456321290981627  loss_variational =  2.3865088283279787
epoch =  0 batch =  163 / 1000 loss_uniform =  2.0436333001025617  loss_variational =  2.3829907822462673
epoch =  0 batch =  164 / 1000 loss_uniform =  2.041665043045835  loss_variational =  2.3793153094082338
epoch =  0 batch =  165 / 1000 loss_uniform =  2.0395518613584116  loss_variational =  2.3756414189483173
epoch =  0 batch =  166 / 1000 loss_uniform =  2.0376053797193325  loss_variational =  2.372096761163458
epoch =  0 batch =  167 / 1000 loss_uniform =  2.0357514441370252  loss_variational =  2.368721665022615
epoch =  0 batch =  168 / 1000 loss_uniform =  2.0339115616821113  loss_variational =  2.3652909000714613
epoch =  0 batch =  169 / 1000 loss_uniform =  2.0320138839574966  loss_variational =  2.3620890936202543
epoch =  0 batch =  170 / 1000 loss_uniform =  2.0

epoch =  0 batch =  239 / 1000 loss_uniform =  1.9325100068766696  loss_variational =  2.1872250500084456
epoch =  0 batch =  240 / 1000 loss_uniform =  1.9314478784799571  loss_variational =  2.185433349510033
epoch =  0 batch =  241 / 1000 loss_uniform =  1.9303882572166153  loss_variational =  2.1836693855736753
epoch =  0 batch =  242 / 1000 loss_uniform =  1.9292671192776067  loss_variational =  2.1820429154664023
epoch =  0 batch =  243 / 1000 loss_uniform =  1.9283219561164757  loss_variational =  2.1802525981463514
epoch =  0 batch =  244 / 1000 loss_uniform =  1.9272636531806377  loss_variational =  2.178476048786131
epoch =  0 batch =  245 / 1000 loss_uniform =  1.9262617763207879  loss_variational =  2.176736729485647
epoch =  0 batch =  246 / 1000 loss_uniform =  1.9252146283785498  loss_variational =  2.1748035797258694
epoch =  0 batch =  247 / 1000 loss_uniform =  1.9242078268576244  loss_variational =  2.17288868099089
epoch =  0 batch =  248 / 1000 loss_uniform =  1.92

...

epoch =  1 batch =  99 / 1000 loss_uniform =  1.629113298473936  loss_variational =  1.6565155802351055
epoch =  1 batch =  100 / 1000 loss_uniform =  1.6291351640224456  loss_variational =  1.6563155877590179
epoch =  1 batch =  101 / 1000 loss_uniform =  1.629168196479873  loss_variational =  1.656421954088872
epoch =  1 batch =  102 / 1000 loss_uniform =  1.6292017639852037  loss_variational =  1.656356372085272
epoch =  1 batch =  103 / 1000 loss_uniform =  1.6291343714427022  loss_variational =  1.656329342462484
epoch =  1 batch =  104 / 1000 loss_uniform =  1.6292056017197096  loss_variational =  1.6563335760281637
epoch =  1 batch =  105 / 1000 loss_uniform =  1.629287959280468  loss_variational =  1.6562902518681117
epoch =  1 batch =  106 / 1000 loss_uniform =  1.6293926463936859  loss_variational =  1.6561358109960016
epoch =  1 batch =  107 / 1000 loss_uniform =  1.6294617942560499  loss_variational =  1.6561722532611027
epoch =  1 batch =  108 / 1000 loss_uniform =  1.6294

epoch =  1 batch =  177 / 1000 loss_uniform =  1.629983339606032  loss_variational =  1.654963948632364
epoch =  1 batch =  178 / 1000 loss_uniform =  1.629958206348205  loss_variational =  1.6549839678775056
epoch =  1 batch =  179 / 1000 loss_uniform =  1.6299071964604894  loss_variational =  1.6549861198031033
epoch =  1 batch =  180 / 1000 loss_uniform =  1.6299321631590524  loss_variational =  1.655004576179716
epoch =  1 batch =  181 / 1000 loss_uniform =  1.6298844037135003  loss_variational =  1.6549438995551007
epoch =  1 batch =  182 / 1000 loss_uniform =  1.6298224670546395  loss_variational =  1.654971116191738
epoch =  1 batch =  183 / 1000 loss_uniform =  1.6298347705048941  loss_variational =  1.6550593043937054
epoch =  1 batch =  184 / 1000 loss_uniform =  1.629820797106494  loss_variational =  1.6550456194773961
epoch =  1 batch =  185 / 1000 loss_uniform =  1.629782440211322  loss_variational =  1.6549670122765203
epoch =  1 batch =  186 / 1000 loss_uniform =  1.6297

epoch =  1 batch =  255 / 1000 loss_uniform =  1.6294018932417327  loss_variational =  1.6544519073822916
epoch =  1 batch =  256 / 1000 loss_uniform =  1.629426009953022  loss_variational =  1.6544656865298746
epoch =  1 batch =  257 / 1000 loss_uniform =  1.6293736779736174  loss_variational =  1.6544684385047346
epoch =  1 batch =  258 / 1000 loss_uniform =  1.6293517600658327  loss_variational =  1.6544117904448692
epoch =  1 batch =  259 / 1000 loss_uniform =  1.6293591734985586  loss_variational =  1.654409821889575
epoch =  1 batch =  260 / 1000 loss_uniform =  1.6293446059410388  loss_variational =  1.6544238833280709
epoch =  1 batch =  261 / 1000 loss_uniform =  1.6293398763028142  loss_variational =  1.6544518333741987
epoch =  1 batch =  262 / 1000 loss_uniform =  1.629299983723473  loss_variational =  1.6544118218749533
epoch =  1 batch =  263 / 1000 loss_uniform =  1.6292881820591683  loss_variational =  1.654402312217103
epoch =  1 batch =  264 / 1000 loss_uniform =  1.6

epoch =  1 batch =  333 / 1000 loss_uniform =  1.6294577397383734  loss_variational =  1.6536214888632834
epoch =  1 batch =  334 / 1000 loss_uniform =  1.629475187041803  loss_variational =  1.6536187700882643
epoch =  1 batch =  335 / 1000 loss_uniform =  1.6294726122671106  loss_variational =  1.6536121364849716
epoch =  1 batch =  336 / 1000 loss_uniform =  1.6294883737961459  loss_variational =  1.6536008027337847
epoch =  1 batch =  337 / 1000 loss_uniform =  1.6295248238198488  loss_variational =  1.6536014688474843
epoch =  1 batch =  338 / 1000 loss_uniform =  1.6294850460171  loss_variational =  1.6535276650677067
epoch =  1 batch =  339 / 1000 loss_uniform =  1.6294826803657516  loss_variational =  1.6535806961819135
epoch =  1 batch =  340 / 1000 loss_uniform =  1.6294774458688854  loss_variational =  1.6535287888611063
epoch =  1 batch =  341 / 1000 loss_uniform =  1.629498542928277  loss_variational =  1.6535185425162664
epoch =  1 batch =  342 / 1000 loss_uniform =  1.62

KeyboardInterrupt: 

In [ ]:
n_sample = 100000
with torch.no_grad():
    x_uniform = flow_uniform.sample(n_sample).cpu().numpy()
    x_variational = flow_variational.sample(n_sample).cpu().numpy()
x_data_plot = x_data.cpu()[:n_sample,:].numpy()

In [ ]:
bins = np.linspace(-0.5, drop_indices.shape[0]+0.5, drop_indices.shape[0]+2)
plt.hist(np.sum(x_data_plot == 0, axis=1), histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins = bins)
plt.hist(np.sum(x_uniform == 0, axis=1), edgecolor="red", histtype="step", bins = bins)
plt.hist(np.sum(x_variational == 0, axis=1), edgecolor="green", histtype="step", bins = bins)
plt.show()

In [ ]:
bins = np.linspace(0, drop_indices.shape[0], 20)
plt.hist(np.sum(x_data_plot, axis=1), histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins = bins)
plt.hist(np.sum(x_uniform, axis=1), edgecolor="red", histtype="step", bins = bins)
plt.hist(np.sum(x_variational, axis=1), edgecolor="green", histtype="step", bins = bins)
plt.show()

In [ ]:
bins = np.linspace(0, 1, 20)
plt.hist(x_data_plot[:,2], histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins = bins)
plt.hist(x_uniform[:,2], edgecolor="red", histtype="step", bins = bins)
plt.hist(x_variational[:,2], edgecolor="green", histtype="step", bins = bins)
plt.show()